In [23]:
# !pip install pygame

In [26]:
import mindwave, time
from pprint import pprint
import socket,select
import time, datetime, sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygame import mixer
import pathlib

In [2]:
secondsToSample = 30
Fs=128 
show=False
samplepoints = Fs*secondsToSample

In [3]:
print("Connecting")
headset = mindwave.Headset('/dev/tty.MindWaveMobile-SerialPo')
time.sleep(2)
print("Connected!")

Connecting
Connected!


In [4]:
def on_raw(headset, rawvalue):
    (eeg, attention) = (headset.raw_value, headset.attention)
    
    ts = time.time()
    data['timestamp'].append(ts)
    data['raw_value'].append(eeg)
    data['attention'].append(attention)

    global currentTimestamp
    global currentRawValue
    global currentAttention
    currentTimestamp = ts
    currentRawValue = eeg
    currentAttention = attention

In [5]:
data = dict()
data['timestamp'] = list()
data['raw_value'] = list()
data['attention'] = list()
currentTimestamp = None
currentRawValue = None
currentAttention = None

sampled_data = dict()
sampled_data['timestamp'] = list() 
sampled_data['raw_value'] = list()
sampled_data['attention'] = list()

filename = './data/myRawEEG.csv'
filename_listener = './data/myRawEEG_listener.csv'

In [ ]:
try:
    while (headset.poor_signal > 5):
        print("Headset signal noisy %d. Adjust the headset and the earclip." % (headset.poor_signal))
        time.sleep(0.1)
        
    print("Writing %d seconds output to %s" % (secondsToSample,filename))
    stime = time.time()
    headset.raw_value_handlers.append( on_raw )
    prevTime = 0
    while ((time.time()-stime)<secondsToSample):
        if headset.poor_signal > 5 :
            print("Headset signal noisy %d. Adjust the headset and the earclip." % (headset.poor_signal))

        if currentTimestamp is not None: 
            sampled_data["timestamp"].append(currentTimestamp)
            sampled_data["raw_value"].append(currentRawValue)
            sampled_data["attention"].append(currentAttention)

        timeDiff = int(time.time()-stime)
        if(timeDiff != prevTime) : 
            print("seconds elapsed: " + str(timeDiff))
            prevTime = timeDiff
        time.sleep(1/Fs)
        pass

finally:
    
    df = pd.DataFrame.from_dict(sampled_data)
    df.to_csv(filename, index=False)

    headset.stop()

In [25]:
mixer.init()
mixer.music.load('./data/hip_hop/breathe_and_stop.mp3')
mixer.music.play()

In [36]:
# grab all the music
currentDirectory = pathlib.Path('./data/hip_hop')
currentPattern = "*.mp3"
songs = []
for currentFile in currentDirectory.glob(currentPattern):
    songs.append(str(currentFile))
currentDirectory = pathlib.Path('./data/classical')
currentPattern = "*.mp3"
for currentFile in currentDirectory.glob(currentPattern):
    songs.append(str(currentFile))

In [38]:
len(songs)

35